In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install langchain==0.3.0 langchain-community==0.3.0 langchain-core==0.3.0 langchain-openai langchain-text-splitters faiss-cpu

In [ ]:
# 재시작하고 실행하기
!pip install git+https://github.com/openai/whisper.git

In [2]:
from google.colab import userdata
from openai import OpenAI
import json, os

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, ToolMessage
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.tools import tool
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

from typing import Any, List, Optional, Dict
from langchain_core.caches import BaseCache
from langchain_core.callbacks import Callbacks
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor

import torch
import whisper
from transformers import WhisperModel, WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor
import librosa
import torch.nn.functional as F

try:
    ChatOpenAI.model_rebuild()
except Exception:
    pass

api_key = userdata.get("OPEN_AI")
client = OpenAI(api_key=api_key)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# tool
# 1. ASR
# 2. 병 분리
# 3. RAG

# tool
1. 화자 분리 ASR
2. 병 구분
3. 화자 텍스트 + 병 구분 + 자가진단표 정보로 정상, 관찰, 주의, 위험 구분
4. 리포트 작성

## 1. 화자 분리
- 화자 분리와 타임 스탬프 asr에 담는다
- 사용 모델: gpt-4o-transcribe-diarize
- 반환값은 연속하여 작성

In [3]:
# 1. 화자 분리 ASR
import json
from langchain_core.tools import tool

@tool
def diarized_transcription_tool(audio_path: str) -> dict:
    """
    오디오 파일 경로를 받아 텍스트로 변환(ASR)합니다.
    화자 분리 없이 전체 대화 내용을 텍스트로 반환합니다.
    """
    try:
        with open(audio_path, "rb") as audio_file:
            # 가장 안정적인 기본 모델 사용 (whisper-1)
            transcript = client.audio.transcriptions.create(
                model="gpt-4o-transcribe",
                file=audio_file,
                response_format="text"  # 복잡한 JSON 대신 텍스트만 바로 받습니다.
            )

        # transcript는 문자열(str)로 반환됩니다.
        return {"text": transcript}

    except Exception as e:
        print(f"ASR Tool Error: {e}")
        return {"text": f"음성 인식 실패: {str(e)}"}

# transcript는 sp0, sp2 이렇게 나오고 나중에 transcript를 몽땅 넣을 때는 둘 중에 한 명은 보호자, 피보호자이며, 대화의 내용에 뇌졸중, 치매, 파킨슨병, 정상 중 하나의 증상을 피보호자가 갖고 있으니까 이 점을 고려하라고 해야 할 듯

## 2. 병 분류
1. 사용 모델: 학습시킨 openai/whisper-tiny (편리상 classifier layer없이 구분)
2. 정확도: 88% 이상
3. 구분: 뇌졸중, 기타 및 복합 퇴행성 뇌질환, 정상
4. 출력: 리스트에 [뇌졸중 확률, 퇴행성 뇌질환 확률, 정상 확률]로 출력 (모두 float으로 소수점 4자리까지 출력되도록 함)

In [ ]:
# 모델 불러오기
model_name = 'openai/whisper-tiny'
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
processor = WhisperProcessor.from_pretrained(model_name, language='Korean')
tokenizer = WhisperTokenizer.from_pretrained(model_name, language='Korean')
whisper_model = whisper.load_model(model_name.split('-')[-1])

device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("tiny").to(device)
ckpt = torch.load("/content/drive/MyDrive/코딩/새싹해커톤/whisper_tiny_cls.pt", map_location=device)
model.load_state_dict(ckpt["model_state_dict"], strict=False)

model.eval()

def classify_audio(audio_path):
    global model
    audio, _ = librosa.load(audio_path, sr=16000)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(device)

    audio_features = model.encoder(mel.unsqueeze(0))

    bos = tokenizer.bos_token_id
    decoder_input_ids = torch.tensor([[bos]], device=device)

    with torch.no_grad():
        logits = model.decoder(decoder_input_ids, audio_features)

    # 앞 3개만 선택
    logits3 = logits[:, -1, :3]          # (1,3)
    probs = F.softmax(logits3, dim=-1)   # (1,3)

    probs = probs.detach().cpu().numpy().flatten()
    formatted_probs = [round(float(p), 4) for p in probs]
    percent_probs = [round(float(p) * 100, 2) for p in formatted_probs]

    return percent_probs

In [5]:
@tool
def classify_neuro_status_tool(audio_path: str) -> dict:
    '''
    질병 분류
    '''
    probs = classify_audio(audio_path)
    return {"accuracy": probs}

## 3. RAG
1. 랭체인 기반 RAG
2. 임베딩 모델: text-embedding-3-small
3. 역할: 문서에서 적절한 정보를 찾고, 이에 대한 일반인이 이해하기 쉬운 설명 제공
4. 설명 시 사용 모델: gpt-4o-mini

In [6]:
class DiseaseRAG:
    """LangChain 기반 질병 정보 RAG 클래스"""

    def __init__(self, api_key: str):
        self.api_key = api_key
        # OpenAI 임베딩 모델 사용
        self.embeddings = OpenAIEmbeddings(
            model="text-embedding-3-small",
            openai_api_key=api_key
        )
        self.vectorstore = None
        self.llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.7,
            openai_api_key=api_key
        )

    def load_and_index_document(self, file_path: str, chunk_size: int = 500, chunk_overlap: int = 100):
        """
        문서를 로드하고 청크로 분할한 뒤 FAISS 벡터스토어에 인덱싱

        Args:
            file_path: 의료 문서 txt 파일 경로
            chunk_size: 청크 크기 (문자 수)
            chunk_overlap: 청크 간 겹치는 부분 (문자 수)
        """
        # 문서 로드
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()

        # 텍스트 분할기 설정
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            separators=["\n\n", "\n", " ", ""]
        )

        # 텍스트를 청크로 분할
        chunks = text_splitter.split_text(text)

        # FAISS 벡터스토어 생성 및 인덱싱
        self.vectorstore = FAISS.from_texts(
            texts=chunks,
            embedding=self.embeddings
        )

        print(f"문서 인덱싱 완료: {len(chunks)}개 청크")
        return len(chunks)

    def search_and_simplify(self, query: str, k: int = 3):
        """
        쿼리로 관련 문서를 검색하고 GPT로 쉬운 설명 생성

        Args:
            query: 검색 쿼리 (질병명 등)
            k: 검색할 문서 개수

        Returns:
            보호자가 이해하기 쉬운 설명 텍스트
        """
        if self.vectorstore is None:
            return "RAG가 초기화되지 않았습니다."

        # 유사도 검색으로 관련 문서 가져오기
        docs = self.vectorstore.similarity_search(query, k=k)
        retrieved_text = "\n\n".join([doc.page_content for doc in docs])

        # 프롬프트 템플릿 정의
        prompt_template = """다음은 '{query}'에 대한 의료 문서에서 검색한 내용입니다.
이 내용을 보호자가 이해하기 쉬운 한국어 설명으로 바꿔 주세요.

### 검색된 내용:
{context}

### 요구사항:
1. 전문 용어를 일상 언어로 바꾸기
2. 간단하고 명확하게 설명
3. 중요 정보는 포함하되 너무 길지 않게
4. 친근하고 안심시키는 톤으로 작성
5. 2-3문단 정도로 간결하게

### 양식 예시:
뇌혈관이 막히거나 터져서 생기는 질환으로, 말이 꼬이거나 한쪽이 약해지는 증상이 동반될 수 있습니다.
평소 고혈압, 당뇨, 심장질환이 있으면 위험이 높아집니다.
자가진단표의 혈압/어지럼 응답, 한쪽 팔이 마비됨 문항  체크 때문에 높음으로 평가되었습니다.

###설명: """

        prompt = PromptTemplate(
            template=prompt_template,
            input_variables=["query", "context"]
        )

        # LLM으로 쉬운 설명 생성
        formatted_prompt = prompt.format(query=query, context=retrieved_text)
        response = self.llm.invoke(formatted_prompt)

        return response.content

    def get_qa_chain(self):
        """
        RetrievalQA 체인 반환 (선택적 사용)
        """
        if self.vectorstore is None:
            raise ValueError("벡터스토어가 초기화되지 않았습니다.")

        qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vectorstore.as_retriever(search_kwargs={"k": 3})
        )

        return qa_chain

In [7]:
_rag_instance = None
# rag용 의료 문서
# 주소 변경 필


def initialize_disease_rag(api_key: str, document_path: str):
    """
    RAG 초기화 및 문서 인덱싱 (노트북 시작 시 한 번만 실행)

    Args:
        api_key: OpenAI API 키
        document_path: 의료 문서 txt 파일 경로
    """
    global _rag_instance
    _rag_instance = DiseaseRAG(api_key=api_key)
    _rag_instance.load_and_index_document(document_path)
    print("RAG 시스템 준비 완료")

In [8]:
@tool
def retrieve_disease_info_tool(query: str) -> dict:
    """
    특정 질병에 대한 의학 문서 컨텍스트를 RAG 방식으로 가져옵니다.
    질병명이나 증상을 입력하면 OpenAI 임베딩으로 관련 정보를 검색하고,
    GPT로 보호자가 이해하기 쉬운 설명을 생성합니다.

    Args:
        query: 질병명 또는 검색 쿼리 (예: "뇌졸중", "파킨슨병", "치매", "루게릭병")

    Returns:
        보호자가 이해하기 쉬운 설명이 담긴 딕셔너리
    """
    global _rag_instance
    context = _rag_instance.search_and_simplify(query, k=3)

    return {"context": context}

In [9]:
# 백에서 받아야 하는 거: 자가문단표, 음성, 환자 정보
# 보내야 하는 거: 백으로 출력값

## 4. AI Agent
1. funciton calling
2. 프롬프트
3. json 형식으로 전체 결과 출력되도록
4. ai agent: 병에 대한 위험도 판단(뇌졸중, 치매, 파킨슨병, 루게릭병)

In [10]:
# 4. function calling
agent_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
당신은 뇌졸중 치매, 파킨슨병, 루게릭병을 평가하는
AI 의료 보조 에이전트이다.

### 사용 가능한 tool:
- diarized_transcription_tool(audio_path): ASR
- classify_neuro_status_tool(audio_path): 음성을 기준으로 뇌질환을 판별
- retrieve_disease_info_tool(query): 특정 질병에 대한 의학 문서 컨텍스트를 RAG 방식으로 가져옴

### 최종 목적:
사용자가 제공한 정보(음성 파일 경로, 자가 문진표 정보)를 바탕으로
다음과 같은 Python 딕셔너리 형태의 JSON을 생성하는 것이다.

result = {{
  "accuracy": [float(뇌졸중 확률), float(퇴행성 뇌질환 확률), float(문제 없음 확률)],
  "ASR": "통화 전사 데이터",
  "risk": ["뇌졸중 위험도", "치매 위험도", "파킨슨병 위험도", "루게릭병 위험도"],
  "explain": ["뇌졸중 설명", "치매 설명", "파킨슨병 설명", "루게릭병 설명"]
}}

### 제약 조건:
- "accuracy"는 classify_neuro_status_tool 툴의 결과를 그대로 사용한다.
- "ASR"에는 diarized_transcription_tool을 사용해 얻은 전체 결과를 절대 요약하거나 내용을 변경하지 않은 채로 넣는다.
- "risk" 리스트는 반드시 길이 4이며, 순서는 [뇌졸중, 치매, 파킨슨병, 루게릭병] 이다.
- 각 위험도 값은 "정상", "관찰", "주의", "위험" 중 하나여야 한다. 이때 판단은 accuracy, ASR, 자가문단표를 기준으로 판단해야 한다.
- "explain" 리스트는 길이 4이며, 순서 역시 [뇌졸중, 치매, 파킨슨병, 루게릭병] 이다.
- 각 설명은 보호자가 이해하기 쉬운 한국어로 작성한다.
- 만약 해당 질병 위험도가 "정상"인 경우에는 설명은 작성하지 않는다.
- 최종 응답은 반드시 위 result 딕셔너리 형태와 동일한 구조의 JSON 객체로만 출력한다.
  그 외의 텍스트(설명, 사족)는 출력하지 않는다.

### tool을 사용할 때:
1) 문단표 정보를 통해 현 상태에 대한 정보를 받는다. 문단표의 점수는 1~5 사이로, 1은 전혀 그렇지 않다, 5는 매우 그렇다를 나타낸다.
2) 그 다음 classify_neuro_status_tool으로 세 가지 범주 확률을 얻는다.
3) diarized_transcription_tool로 보호자와 피보호자의 대화 정보를 얻는다.
4) 문단표 정보, 2번의 세 가지 범주 확률, 보호자와 피호자의 대화 내용을 기준으로 뇌졸중, 치매, 파킨슨병, 루게릭병에 대한 위험도를 정상, 관찰, 주의, 위험으로 각각 판단한다.
5) 관찰 이상 단계의 병에 대한 정보를 retrieve_disease_info_tool을 사용해서 각 질병에 대한 설명을 보완하여 보호자에게 전달할 설명을 구성한다. 이때, 자가문단표의 내용을 그대로 출력하기 보다 보호자가 인지하고 있어야 할 내용이나 보호자가 수행해야 할 내용을 중심으로 출력한다. 출력 시, 마침표와 쉼표만 특수문자로 사용한다. 문장 종결 시, 마침표를 사용하며, 쉼표는 문장 내에서만 사용한다.
"""
    ),
    (
        "user",
        """
다음 정보를 바탕으로 result를 생성해 주세요. self_report는 질병에 적합한 질문과 그에 대한 정도를 반영한 숫자가 포함된 정보입니다.

- audio_path: {audio_path}
- self_report(JSON): {self_report_json}

위 audio_path를 사용해 tools를 호출해서 ASR을 생성하고 분류 확률을 계산하고,
자가 문진표 정보를 반영하여 최종 result를 만드세요.
"""
    ),
    # [수정 3] Agent가 생각할 공간(Scratchpad) 추가 (이거 없으면 ValueError 남)
    ("placeholder", "{agent_scratchpad}"),
])

# 에이전트 생성
# (tools 리스트는 위에서 이미 정의되어 있다고 가정합니다)
tools = [
    diarized_transcription_tool,
    classify_neuro_status_tool,
    retrieve_disease_info_tool,
]

agent_llm = ChatOpenAI(
    # [수정 4] 존재하지 않는 모델명(gpt-4.1-nano) -> gpt-4o-mini로 변경
    model="gpt-5.1",
    temperature=0.7,
    openai_api_key=api_key,
)

agent = create_tool_calling_agent(agent_llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


# 파이프라인
def run_agent_with_function_calling(audio_path: str, self_report: dict) -> dict:
    """
    LangChain tool-calling 에이전트를 사용하여 전체 파이프라인을 수행하고,
    최종적으로 result 딕셔너리를 반환한다.
    """
    user_input = {
        "audio_path": audio_path,
        "self_report_json": json.dumps(self_report, ensure_ascii=False),
    }

    # agent 실행
    output = agent_executor.invoke(user_input)

    # agent의 최종 출력 가져오기
    raw = output.get("output", output)

    # [수정 5] 결과 파싱 강화 (Markdown 코드 블록 제거 기능 추가)
    if isinstance(raw, str):
        # LLM이 ```json ... ``` 형태로 줄 경우를 대비해 태그 제거
        clean_raw = raw.replace("```json", "").replace("```", "").strip()
        try:
            result = json.loads(clean_raw)
        except Exception:
            print("JSON 파싱 실패. 원본 출력:", raw)
            raise ValueError(f"JSON 파싱 실패: {raw}")
    else:
        # 혹시 dict로 바로 나왔다면 그대로 사용
        result = raw

    # 필수 키 검증
    required_keys = ["accuracy", "ASR", "risk", "explain"]
    for k in required_keys:
        if k not in result:
            raise ValueError(f"결과에 '{k}' 키가 없습니다. 실제 결과: {result}")

    return result

In [ ]:
# 1. RAG 초기화 (한 번만 실행)
initialize_disease_rag(api_key=api_key, document_path="/content/drive/MyDrive/코딩/새싹해커톤/rag_practicce.txt")

# 2. 에이전트 실행
# audio_path = "/content/drive/MyDrive/코딩/새싹해커톤/일반남여_일반통합06_F_1536505292_35_경상_실내_06320.wav"
audio_path = "/content/drive/MyDrive/코딩/새싹해커톤/talk_set2_collectorgs384_speakergs4917_speakergs4918_6_0_120.wav"
self_report = {
    "한쪽 얼굴이 둔하고 손발이 저리거나 힘이 빠진다": 1,
    "한쪽 손에 힘이 없어 물건을 떨어뜨리거나 다리가 후들거려 비틀거린다": 1,
    "어떤 일이 언제 일어났는지 기억하지 못할 때가 있다": 2,
    "며칠 전에 들었던 이야기를 잊는다": 2,
    "손을 움직이거나 가만히 있을 때도 손이 떨린다": 1,
    "침대나 의자에서 일어날 때 몸이 무겁고 힘들다": 2,
    "옷 단추를 잠그거나 물건을 잡기 힘들다": 1,
    "근육 경련이 일어난다": 2
}

result = run_agent_with_function_calling(audio_path, self_report)
print(json.dumps(result, ensure_ascii=False, indent=2))